In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\AutoSummaryAI\\AutoSummaryAI\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sahil\\OneDrive\\Desktop\\AutoSummaryAI\\AutoSummaryAI'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from AutoSummaryAI.constants import *
from AutoSummaryAI.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm
import os
from pathlib import Path

c:\Users\sahil\anaconda3\envs\autosumai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-01 23:48:31,412: INFO: config: PyTorch version 2.6.0+cu126 available.]


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    
    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Fix: Load model and tokenizer from local file paths instead of treating them as repo IDs
        print(f"Loading tokenizer from: {self.config.tokenizer_path}")
        print(f"Loading model from: {self.config.model_path}")
        
        # Check if paths exist
        if not os.path.exists(self.config.tokenizer_path):
            raise FileNotFoundError(f"Tokenizer path not found: {self.config.tokenizer_path}")
        if not os.path.exists(self.config.model_path):
            raise FileNotFoundError(f"Model path not found: {self.config.model_path}")
            
        # Load tokenizer and model using local file paths
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path, local_files_only=True)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path, local_files_only=True).to(device)
       
        # Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load_metric('rouge', trust_remote_code=True)

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], 
            rouge_metric, 
            model_pegasus, 
            tokenizer, 
            batch_size=2, 
            column_text='dialogue', 
            column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)
        
        print(f"Evaluation completed. Metrics saved to {self.config.metric_file_name}")
        return df

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    raise e

[2025-05-01 23:48:32,178: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-01 23:48:32,181: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-01 23:48:32,182: INFO: common: created directory at: artifacts]
[2025-05-01 23:48:32,183: INFO: common: created directory at: artifacts/model_evaluation]
Loading tokenizer from: artifacts/model_trainer/tokenizer
Loading model from: artifacts/model_trainer/pegasus-samsum-model-lora


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\sahil\AppData\Local\Temp\ipykernel_3104\393949666.py:64: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge', trust_remote_code=True)
100%|██████████| 5/5 [03:31<00:00, 42.25s/it]

[2025-05-01 23:52:16,240: INFO: rouge_scorer: Using default tokenizer.]
Evaluation completed. Metrics saved to artifacts/model_evaluation/metrics.csv
